In [1]:
import sys
import warnings
warnings.simplefilter('ignore')
sys.path.append('..')

In [2]:
from data.custom_reader.credit_reader import Reader
from data.custom_spliter.normal_spliter import Spliter
from data.data_loader import DataLoader

from models.base_model import Model
from models.custom_model.xgb_model import XGB

from methods.kfold import KFoldEnsemble

from eval.custom_evaler.auc_evaler import Evaler
from submit.custom_submitter.credit_submitter import Submitter

ModuleNotFoundError: No module named 'xgboost'

In [18]:
config = {
    "print_every": 50,
    "param": {
        'learning_rate': 0.05,
        'max_depth': 7,
        'num_leaves': 24,
        'feature_fraction': 0.6,
        'bagging_fraction': 0.8,
        'bagging_freq': 1,
        'objective': 'binary',
        'save_binary': True,
        'metric': 'auc',
        'verbose': -1,
        'seed': 0,
        'num_iterations': 500,
        # 'subsample_freq': 5,
        'early_stopping_round' : 300,
        'num_threads': 12,
    }
}

In [19]:
# load data
custom_reader = Reader('../demo/credit_data', 'train.pkl', 'train_target.pkl')
custom_spliter = Spliter()
data = DataLoader(custom_reader, custom_spliter)
data.load()

In [20]:
# initialize model
lgb_custom = LGB(config)
base_model = Model(lgb_custom)

In [21]:
# initialize metric
evaler = Evaler()

In [22]:
# intialize method
kfoldEnsemble = KFoldEnsemble(base_model=base_model, evaler=evaler, nfold=5, seed=0, nni_log=False)

In [23]:
# start training
kfoldEnsemble.fit(data)

[KFold Ensemble Input] X:  (132029, 103) y:  (132029, 1)


LightGBMError: Unknown objective type name: rank:pairwise

In [27]:
# initialize submitter
submitter = Submitter(submit_file_path='../demo/credit_data/submit.csv', save_path='../demo', file_name='lgb_base.csv')

In [ ]:
# submit your prediction
submitter.submit(kfoldEnsemble, data)